In [64]:
import drift_algorithms as algo
import numpy as np 
import json
import emip_toolkit as EMTK
import pandas as pd
import correction
import tensorflow as tf
import prep_GB_img as prepgb

## RS81

In [65]:
dirty_file_names = ["S_8152_S1_TEX.json", "S_8063_S1_TEX.json",
              "S_8107_S1_TEX.json", "S_8331_S1_TEX.json"]
clean_file_names = ["S_8152_S1_TEX_CORRECTED.json", "S_8063_S1_TEX_CORRECTED.json",
              "S_8107_S1_TEX_CORRECTED.json", "S_8331_S1_TEX_CORRECTED.json"]
image_path = "Golden_Set/R8S1/"
image = "TEX_R8S1_bg.png"
file_path = "Golden_Set/R8S1/"
txt_file_name = "TEX_R8S1_text.txt"
file_path = "Golden_Set/R8S1/"

In [66]:
aoi = EMTK.find_aoi(image, image_path, level="sub-line")
aois_with_tokens = EMTK.add_tokens_to_AOIs(file_path,txt_file_name, aoi)
line_ys = correction.find_lines_Y(aois_with_tokens)

In [67]:
word_centers = correction.find_word_centers(aoi)
word_centers = np.array(word_centers.copy(), dtype=int)

duration_word_centers = correction.find_word_centers_and_duration(aois_with_tokens)
duration_word_centers = np.array(duration_word_centers.copy(), dtype=int)

In [68]:
model = tf.keras.models.load_model("saved_models_mix_ayn/model_tl.h5")

In [54]:
acc = [[], [], [], [], [], [], [], [], []]

for i in range(4):
    with open(file_path + dirty_file_names[i]) as f:
       dirty_dict = json.load(f)
    
    #dirty dataframe
    dirty_df = pd.DataFrame.from_dict(dirty_dict)
    #Transpose the dirty
    dirty_df = pd.DataFrame.transpose(dirty_df)
    #convert to numpy
    dirty_np = np.array(dirty_df)

   # clean, corrected data
    with open(file_path + clean_file_names[i]) as f:
       clean_dict = json.load(f)

    clean_fixs = clean_dict["fixations_data"]
    clean_df = pd.DataFrame.from_dict(clean_fixs)
    clean_df = pd.DataFrame.transpose(clean_df)
    clean_np = np.array(clean_df)
    
    #run the algorithms
    
    #attach
    np_array = np.array(dirty_np.copy())
    correction_p = algo.attach(np_array, line_ys)
    percentage, match_list = correction.correction_quality(aoi, clean_np, correction_p)
    acc[0].append(percentage)
   #  correction.draw_correction(image_path+image, correction_p, match_list)
    
    #chain
    np_array = np.array(dirty_np.copy())
    correction_p = algo.chain(np_array, line_ys)
    percentage, match_list = correction.correction_quality(aoi, clean_np, correction_p)
    acc[1].append(percentage)
   #  correction.draw_correction(image_path+image, correction_p, match_list)    
    
    #cluster
    np_array = np.array(dirty_np.copy())
    correction_p = algo.cluster(np_array, line_ys)
    percentage, match_list = correction.correction_quality(aoi, clean_np, correction_p)
    acc[2].append(percentage)
   #  correction.draw_correction(image_path+image, correction_p, match_list)
    
    #merge
    np_array = np.array(dirty_np.copy())
    correction_p = algo.merge(np_array, line_ys, y_thresh = 50)
    percentage, match_list = correction.correction_quality(aoi, clean_np, correction_p)
    acc[3].append(percentage)
   #  correction.draw_correction(image_path+image, correction_p, match_list)
    
    #regress
    np_array = np.array(dirty_np.copy())
    correction_p = algo.regress(np_array, line_ys)
    percentage, match_list = correction.correction_quality(aoi, clean_np, correction_p)
    acc[4].append(percentage)
   #  correction.draw_correction(image_path+image, correction_p, match_list)

    #segment
    np_array = np.array(dirty_np.copy())
    correction_p = algo.segment(np_array, line_ys)
    percentage, match_list = correction.correction_quality(aoi, clean_np, correction_p)
    acc[5].append(percentage)
   #  correction.draw_correction(image_path+image, correction_p, match_list)

    #split
    np_array = np.array(dirty_np.copy())
    correction_p = algo.split(np_array, line_ys)
    percentage, match_list = correction.correction_quality(aoi, clean_np, correction_p)
    acc[6].append(percentage)
   #  correction.draw_correction(image_path+image, correction_p, match_list)
    
    #stretch
    np_array = np.array(dirty_np.copy())
    correction_p = algo.stretch(np_array, line_ys)
    percentage, match_list = correction.correction_quality(aoi, clean_np, correction_p)
    acc[7].append(percentage)
   #  correction.draw_correction(image_path+image, correction_p, match_list)
    
    # warp
    np_array = np.array(dirty_np.copy(), dtype=int)
    durations = np.delete(np_array, 0, 1)
    durations = np.delete(durations, 0, 1)
    np_array = np.delete(np_array, 2, 1)
    warp_correction = algo.warp(np_array, word_centers)
    percentage, match_list = correction.correction_quality(aoi, clean_np, warp_correction)
    acc[8].append(percentage)
   #  correction.draw_correction(image_path+image, correction_p, match_list)

   # with classifier
    np_array = np.array(dirty_np.copy())
    predicted_error = correction.predict_error(
                image, 
                np_array, 
                aois_with_tokens, 
                input_x, 
                input_y, 
                model)
                

In [55]:
print(acc)

[[0.788546255506608, 0.33649289099526064, 0.4423076923076923, 0.6172248803827751], [0.8193832599118943, 0.3412322274881517, 0.5048076923076923, 0.6220095693779905], [0.29955947136563876, 0.3791469194312796, 0.7211538461538461, 0.2583732057416268], [0.09251101321585903, 0.2037914691943128, 0.10576923076923077, 0.11961722488038277], [0.788546255506608, 0.47393364928909953, 0.5625, 0.5502392344497608], [0.02643171806167401, 0.4075829383886256, 0.6826923076923077, 0.2679425837320574], [0.8458149779735683, 0.35071090047393366, 0.4326923076923077, 0.5358851674641149], [0.7929515418502202, 0.009478672985781991, 0.6730769230769231, 0.22966507177033493], [0.5330396475770925, 0.15165876777251186, 0.6490384615384616, 0.5645933014354066]]
